In [1]:
import pymysql, os, copy, json, time, openpyxl
import pandas as pd
import argparse
import numpy as np
import matplotlib.pyplot as plt
import torch
import re
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from kss import kss
from pororo import Pororo
from konlpy.tag import Mecab
import pickle

In [2]:
def _call_db_info():
    return pymysql.connect(
        host = 'trend.cb7jqghocrtb.ap-northeast-2.rds.amazonaws.com',
        port= 3306,
        user = 'root',
        password='ensol2020!',
        db = 'trend',
        charset = 'utf8')
def extract_parenthese(str):
    items_lst = re.findall('\(([^)]+)', str) #extracts string in () 
    newList = [x for x in items_lst if len(x)>=2] # more than 2
    return newList

def extract_quotes(str):
    items_lst = re.findall('"([^"]*)"', str)
    return items_lst

def parentheses_(tmp_input_sent):    
    tmp_input_sent = re.sub(pattern='\(+', repl=' ', string=tmp_input_sent)#tmp_input_sent = re.sub(pattern='\(\(', repl='\(', string=tmp_input_sent)
    tmp_input_sent = re.sub(pattern='\)+', repl=' ', string=tmp_input_sent)#tmp_input_sent = re.sub(pattern='\)\)', repl='\)', string=tmp_input_sent)
    tmp_input_sent = re.sub(pattern=' +', repl=' ', string=tmp_input_sent)
    input_sent = re.sub(pattern='\\\\',   repl='', string=tmp_input_sent)
    return input_sent
    '''
    m = []
    for i in range(0, len(s_m)):
        if s_m[i][1] < e_m[i][0]:
            m.append((s_m[i], e_m[i]))
        else:
            for j in range(i+1, len(e_m)):
                if s_m[i][1] < e_m[j][0]:
                    m.append((s_m[i], e_m[j]))
                    break;

    if len(m) > 0:
        for i in range(0, len(m)):
            if i == 0:
                tmp_sent1.append(input_sent[:m[i][0][0]])
            else :
                tmp_sent1.append(input_sent[m[(i-1)][1][1]:m[i][0][0]])                        
            tmp_sent2.append(input_sent[m[i][0][1]:m[i][1][0]])
        tmp_sent1.append(input_sent[m[-1][1][1]:])    
        return ' '.join(tmp_sent1 + tmp_sent2)
    else:
        return input_sent
    '''
    
def token_generate(sent, tok, MAX_LEN):
    encode_dict = tok.encode_plus(text=sent, 
                                    add_special_tokens=True, max_length=MAX_LEN,
                                    return_token_type_ids=True, padding='max_length', #pad_to_max_length=True,
                                    return_attention_mask=True, truncation=True) # return_tensors='pt',

    input_id = encode_dict['input_ids']
    attention_mask = encode_dict['attention_mask']
    token_type_id = encode_dict['token_type_ids']
    return input_id, attention_mask, token_type_id 

def _read_file(input_file):
    with open(input_file, "r", encoding="utf-8") as f:
        sentences = []
        labels = []
        for line in f:
            split_line = line.strip().split('\t')
            sentences.append(split_line[0])
            labels.append(split_line[1])
        return sentences, labels
    
def eval_input(test, token, args, pad_token_label_id, mask_padding_with_zero = True):

    words = test.split()
    tokens = []
    slot_label_mask = []
    for word in words:
        word = word.strip()
        word_tokens = token.tokenize(word)
        if not word_tokens:
            word_tokens = [unk_token]  # For handling the bad-encoded word
        tokens.extend(word_tokens)
        slot_label_mask.extend([0] + [pad_token_label_id] * (len(word_tokens) - 1))

    # Account for [CLS] and [SEP]
    special_tokens_count = 2
    if len(tokens) > args.max_seq_len - special_tokens_count:
        slot_label_mask = slot_label_mask[:(args.max_seq_len - special_tokens_count)]

    # Add [SEP] token
    slot_label_mask += [pad_token_label_id]
    slot_label_mask = [pad_token_label_id] + slot_label_mask
    padding_length = args.max_seq_len - len(slot_label_mask)
    slot_label_mask = slot_label_mask + ([pad_token_label_id] * padding_length)
    input_id, attention_mask, token_type_id = token_generate(test, token, args.max_seq_len)

    input_ids = torch.tensor(input_id, dtype=torch.long).reshape(1,-1)
    attention_mask = torch.tensor(attention_mask, dtype=torch.long).reshape(1,-1)
    token_type_ids = torch.tensor(token_type_id, dtype=torch.long).reshape(1,-1)
    slot_label_mask = torch.tensor(slot_label_mask, dtype=torch.long).reshape(1,-1)
    
    return input_ids, attention_mask, token_type_ids, slot_label_mask    


In [3]:
from transformers import BertForTokenClassification
from tokenization_kobert import KoBertTokenizer
from transformers import AutoModelForTokenClassification

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu" #"cpu"#
model = AutoModelForTokenClassification.from_pretrained('./save/')#'monologg/kobert')#, num_labels=len(train_label_l1))
token = KoBertTokenizer.from_pretrained('monologg/kobert')
model.to(device)
model.eval()
print("Model Loaded")

args = argparse.Namespace(  
    max_seq_len = 128
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


Model Loaded


In [5]:
conn = _call_db_info()
curs = conn.cursor()
tmp_insert_sql = "select * from content where date >= 20210701"#"select * from word_dic"
curs.execute(tmp_insert_sql)        
tmp_article = pd.DataFrame(curs.fetchall())
conn.commit()
conn.close()

In [6]:
def eval_ft(test_sent, label_lst_, model_, tok, dev, args):
    preds = None
    pad_token = torch.nn.CrossEntropyLoss().ignore_index    
    with torch.no_grad():
        tmp_input, tmp_attention, tmp_token, tmp_slot = eval_input(test_sent, tok, args, pad_token)    
        inputs = {'input_ids':tmp_input.to(device), 'attention_mask':tmp_attention.to(device), 
                  'labels' : None,
                  'token_type_ids': tmp_token.to(device)}
        output = model_(**inputs)
        logits = output[0]

        if preds is None:
            preds = logits.detach().cpu().numpy()
        else:
            preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)

        preds = np.argmax(preds, axis=2)
        slot_label_map = {i : label for i, label in enumerate(label_lst_)}   
    preds_list = []
    for j in range(preds.shape[1]):
        if tmp_slot[0,j] != pad_token:
            preds_list.append(slot_label_map[preds[0][j]])            

    return preds_list

def get_labels(label_path):
    return [label.strip() for label in open(os.path.join(label_path), 'r', encoding='utf-8')]

In [10]:
with open('Test_LGES_sent_210804.dta', 'rb') as a_sent:
    test_sent = pickle.load(a_sent)
with open('Test_LGES_sent_ner_level1_210804.dta', 'rb') as a_sent_l1:
    test_ner_l1 = pickle.load(a_sent_l1)
with open('Test_LGES_label_l1_210804.dta', 'rb') as a_label1:
    test_label_l1 = pickle.load(a_label1)
with open('LGES_label_l1_210804.dta', 'rb') as a_label1:
    train_label_l1 = pickle.load(a_label1)

In [11]:
rslt,acc,score = [],[],[]
for i in range(0, len(test_sent)):
    #print(test_sent[i])
    preds_list = eval_ft(test_sent[i], train_label_l1, model, token, device, args)
    #print(preds_list)
    ner_preds = [x.split('-')[0] if x != 'UNK' else 'O' for x in preds_list]
    ner_true = test_ner_l1[i].split()
    
    #print(ner_preds, ner_true)
    #print(len(ner_preds), len(ner_true))
    tmp_acc = np.sum([1 if p == t else 0 for t, p in zip(ner_true, ner_preds)]) / len(ner_true)
    
    if np.sum([1 for x in ner_true if x == 'O']) == len(ner_true) :
        tmp_f1score = 999
    else :
        tmp_precision = np.sum([1 if x == ner_true[i] else 0 for i,x in enumerate(ner_preds) if x != 'O' ]) / np.sum([1 for x in ner_preds if x != 'O' ])
        tmp_recall = np.sum([1 if x == ner_preds[i] else 0 for i,x in enumerate(ner_preds) if x != 'O' ]) / np.sum([1 for x in ner_true[:len(ner_preds)] if x != 'O' ])
        tmp_f1score = 2 * (tmp_recall * tmp_precision ) / (tmp_recall + tmp_precision)

        #print('Acc & F1 Score : %.3f / %.3f'%(tmp_acc, tmp_f1score))

    line = ""
    for w, p in zip(test_sent[i].split(), preds_list):
        #line = line + " {}".format(p)
        if p == 'UNK':
            line = line + w + " "
        else :
            line = line + "{}[{}] ".format(w, p)
    rslt.append(line)
    acc.append(tmp_acc)
    score.append(tmp_f1score)
score1 = [s for s in score if s < 999]

<ipython-input-11-9cb49c8f24cf>:16: RuntimeWarning: invalid value encountered in double_scalars
  tmp_precision = np.sum([1 if x == ner_true[i] else 0 for i,x in enumerate(ner_preds) if x != 'O' ]) / np.sum([1 for x in ner_preds if x != 'O' ])
<ipython-input-11-9cb49c8f24cf>:17: RuntimeWarning: invalid value encountered in double_scalars
  tmp_recall = np.sum([1 if x == ner_preds[i] else 0 for i,x in enumerate(ner_preds) if x != 'O' ]) / np.sum([1 for x in ner_true[:len(ner_preds)] if x != 'O' ])
<ipython-input-11-9cb49c8f24cf>:17: RuntimeWarning: divide by zero encountered in true_divide
  tmp_recall = np.sum([1 if x == ner_preds[i] else 0 for i,x in enumerate(ner_preds) if x != 'O' ]) / np.sum([1 for x in ner_true[:len(ner_preds)] if x != 'O' ])
<ipython-input-11-9cb49c8f24cf>:18: RuntimeWarning: invalid value encountered in double_scalars
  tmp_f1score = 2 * (tmp_recall * tmp_precision ) / (tmp_recall + tmp_precision)


In [12]:
np.mean(acc), np.mean(score1)

(0.9605276644991938, 0.958747664618813)

In [13]:
rslt

['M 오토데일리 최태인 기자 볼보자동차가 지난 30일 스웨덴 고텐버그에서 개최된 테크 모멘트 Volvo Cars Tech Moment 이벤트를 통해 글로벌 프리미엄 전기차[제품-B] 시장 선두기업으로의 도약을 위한 미래기술 로드맵을[이벤트-B] 발표했다 ',
 '이번 행사는 2030년 전기차[제품-B] 회사로의 전면적인 전환을 계획하고 있는 볼보자동차의 미래 기술을 최초로 선보이는 자리로 구글 Google 루미나 Luminar 노스볼트 Northvolt 엔비디아 NVIDIA 등 주요 파트너사들 대표들이 참가한 가운데 함께 진행됐다 ',
 '이를 통해 볼보자동차는 완전한 전기차[제품-B] 회사로 전환을 위한 배터리 공급 계획 자체 소프트웨어 개발 확대를 위한 계획 컴퓨팅의[기술-B] 중앙 집중화 차세대 안전 기술에 대한 계획 등 핵심 분야에 대한 세부사항을 공개했다 ',
 '볼보자동차 최고경영자 하칸 사무엘손 Håkan Samuelsson 은 볼보자동차는 오는 2030년까지 프리미엄 순수 전기차[제품-B] 기업으로 전환하기 위해 94년 역사상 가장 최고의 자동차를 만드는 데 전념하고 있다며 지금은 이러한 발전에 있어 흥미로운 시기로 원활한 연결성 업계 최고 수준의 안전표준 그리고 수준 높은 자율주행을 구현하는 순수 전기차에[제품-B] 대한 고객 요구를 만족시킬 것이라고 밝혔다고객 차량의 실시간 데이터를 활용해 새로운 안전 기준 설정볼보자동차는 그 동안 새로운 안전 기술을 개발하는데 있어 실제 도로 ',
 '차세대 볼보자동차는 여기서 나아가 고객이 동의한다면 더욱 안전한 기술을 개발하기 위해 고객의 자동차로부터 실시간 데이터를 전송받아 분석하는 것을 고려하고 있다 ',
 '여기에는 루미나 Luminar 의 고해상도[제품-B] 라이다 ',
 'LiDAR 센서가 수집한 자동차 환경 관련 데이터가 포함된다 ',
 '이를 통해 볼보자동차 엔지니어는 전 세계 수만 명의 운전자의 데이터를 기반으로 테스트 트랙이 아닌 특정 지리적 위치에 대한 자율주행 AD autonomou

In [17]:
pd.DataFrame(rslt).to_csv('Test_Tagging_v210805.csv', encoding='utf-8-sig')